In [29]:
import os
import pandas as pd
import datetime as dt
import numpy as np

In [6]:
print(os.getcwd())

/Users/ZiweiMengyang/Desktop/Python & Machine Learning/Tiger/Capstone/Codes


# Step 1. Load csv into pandas DataFrame (play, down, search)

In [93]:
play = pd.read_csv('../Data/play_sample.csv', header=0)
# if use pyspark 
# play = spark.read.csv('../Data/valid_uid.csv', header=True)

In [94]:
play['play_date'] = pd.to_datetime(play['date'])
play = play.drop(columns=['date'])

In [96]:
print(play.shape)
play.tail()

(386875, 8)


,uid,device,song_id,song_type,play_time,song_length,churn,play_date
386870,167848336.0,ar,6962777.0,0.0,68.0,84.0,0,2017-04-26
386871,167848336.0,ar,22398320.0,0.0,246.0,246.0,0,2017-04-26
386872,167848336.0,ar,22398320.0,0.0,246.0,246.0,0,2017-04-26
386873,167848336.0,ar,22398320.0,0.0,139.0,246.0,0,2017-04-26
386874,167848336.0,ar,80403.0,0.0,153.0,204.0,0,2017-04-27


### Define end_date  ***  Note -  For down and search data, first remove log >= pd.to_date('2017-04-29')

In [78]:
end_date = play['play_date'].max() + dt.timedelta(days=1)
end_date

Timestamp('2017-04-29 00:00:00')

In [21]:
search = pd.read_csv('../Data/search_sample.csv', header=0)
search['search_date'] = pd.to_datetime(search['search_date_str'])
search.dtypes

uid                       float64
search_date_str            object
search_query               object
search_date        datetime64[ns]
dtype: object

In [79]:
search = search[search['search_date'] < end_date]
search.tail()

,uid,search_date_str,search_query,search_date
32970,168541184.0,2017-04-28 22:27:27,%E4%B8%80%E7%94%9F%E6%89%80%E7%88%B1-%28%E7%94...,2017-04-28 22:27:27
32971,168995920.0,2017-04-28 22:34:44,%E4%BC%B4,2017-04-28 22:34:44
32972,168989344.0,2017-04-28 23:18:30,%E6%B4%BB%E7%9D%80+%E9%83%9D%E4%BA%91,2017-04-28 23:18:30
32973,168951008.0,2017-04-28 23:24:02,%E6%B1%9F%E5%8D%97,2017-04-28 23:24:02
32974,168951008.0,2017-04-28 23:24:17,%E6%B1%9F%E5%8D%97,2017-04-28 23:24:17


In [83]:
down = pd.read_csv('../Data/download_sample.csv', header=0)
down.dtypes

uid               float64
song_id_down      float64
song_name_down     object
date_down          object
dtype: object

In [84]:
down['down_date'] = pd.to_datetime(down['date_down'])
down = down[down['down_date'] < end_date].drop(columns=['date_down'])
down.tail()

,uid,song_id_down,song_name_down,down_date
29663,168541184.0,4370025.0,晚风,2017-04-28
29664,168541184.0,135373.0,痛哭的人,2017-04-28
29665,168995920.0,1061665.0,伴(48秒铃声版),2017-04-28
29666,169001472.0,11425031.0,童话镇,2017-04-28
29667,169001472.0,19535292.0,逆流成河,2017-04-28


# Step 2. create Frequency and Avg. Play Time features for play data

In [38]:
def compute_play_freq(df, event, event_date, end_date, window,):
    df_win = df[(pd.to_datetime(end_date) - df[event_date]).dt.days <= window]
    df_feature = df_win.groupby('uid')[event] \
                       .agg(['count', 'mean']) \
                       .rename(columns={'count':'freq_last_' + str(window) + '_days_play', 
                                        'mean':'mean_last_'+ str(window) + '_days_play_time'})
    
    return df_feature

In [39]:
search_1 = compute_play_freq(play, 'play_time', 'play_date', '2017-04-29', 1)

In [44]:
search_1.head(5)

,freq_last_1_days_play,mean_last_1_days_play_time
uid,,
17745172.0,38,109.868421
17972352.0,2,183.500000
19201816.0,32,195.500000
47544816.0,5,79.600000
51908840.0,1,39907.000000


In [45]:
play_1 = compute_play_freq(play, 'play_time', 'play_date', '2017-04-29', 1)
play_3 = compute_play_freq(play, 'play_time', 'play_date', '2017-04-29', 3)
play_7 = compute_play_freq(play, 'play_time', 'play_date', '2017-04-29', 7)
play_14 = compute_play_freq(play, 'play_time', 'play_date', '2017-04-29', 14)
play_30 = compute_play_freq(play, 'play_time', 'play_date', '2017-04-29', 30)

In [100]:
play_feature = pd.merge(play_30, play_14, how='left', left_index=True, right_index=True)
play_feature = pd.merge(play_feature, play_7, how='left', left_index=True, right_index=True)
play_feature = pd.merge(play_feature, play_3, how='left', left_index=True, right_index=True)
play_feature = pd.merge(play_feature, play_1, how='left', left_index=True, right_index=True)

In [101]:
play_feature.head(5)

,freq_last_30_days_play,mean_last_30_days_play_time,freq_last_14_days_play,mean_last_14_days_play_time,freq_last_7_days_play,mean_last_7_days_play_time,freq_last_3_days_play,mean_last_3_days_play_time,freq_last_1_days_play,mean_last_1_days_play_time
uid,,,,,,,,,,
1668933.0,7,53.714286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4754114.0,429,146.522145,171.0,119.216374,90.0,69.766667,35.0,0.000000,NaN,NaN
7950294.0,63,96.174603,59.0,91.203390,38.0,122.447368,25.0,165.880000,NaN,NaN
7950557.0,86,50.476744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10257342.0,151,133.940397,87.0,148.264368,71.0,152.436620,9.0,620.333333,NaN,NaN


# Step 3. Create Frequency features for down and search data respectively

In [50]:
# function for down and search data 
def compute_freq(df, event, event_date, end_date, window,):
    df_win = df[(pd.to_datetime(end_date) - df[event_date]).dt.days <= window]
    df_feature = df_win.groupby('uid')[event_date] \
                       .agg(['count']) \
                       .rename(columns={'count':'freq_last_' + str(window) + '_days_' + event})
                                       
    
    return df_feature

In [102]:
down_feature = compute_freq(down, 'download', 'down_date', end_date, 1)

In [103]:
down_feature.head()

,freq_last_1_days_download
uid,
17745172.0,1
96451488.0,1
150530384.0,1
157725904.0,2
167570816.0,30


In [104]:
for t in [3,7,14,30]:
    down_t = compute_freq(down, 'download', 'down_date', end_date, t)
    down_feature = pd.merge(down_t, down_feature, how='left', left_index=True, right_index=True) 


In [105]:
down_feature.head()

,freq_last_30_days_download,freq_last_14_days_download,freq_last_7_days_download,freq_last_3_days_download,freq_last_1_days_download
uid,,,,,
4754114.0,2,NaN,NaN,NaN,NaN
7950557.0,6,NaN,NaN,NaN,NaN
11771279.0,13,NaN,NaN,NaN,NaN
17745172.0,2,1.0,1.0,1.0,1.0
17972352.0,1,1.0,NaN,NaN,NaN


In [106]:
search_feature = compute_freq(search, 'search', 'search_date', end_date, 1)
for t in [3,7,14,30]:
    search_t = compute_freq(search, 'search', 'search_date', end_date, t)
    search_feature = pd.merge(search_t, search_feature, how='left', left_index=True, right_index=True)



In [107]:
search_feature.head()

,freq_last_30_days_search,freq_last_14_days_search,freq_last_7_days_search,freq_last_3_days_search,freq_last_1_days_search
uid,,,,,
1668933.0,4,NaN,NaN,NaN,NaN
4754114.0,20,4.0,NaN,NaN,NaN
7950557.0,25,NaN,NaN,NaN,NaN
10257342.0,8,1.0,1.0,NaN,NaN
11771279.0,7,NaN,NaN,NaN,NaN


# Step 4. Merge play_feature, search_feature, down_feature together

In [118]:
play_feature_2 = play[['uid', 'device', 'churn']].drop_duplicates()
play_feature_2.shape
# Hypothesis: users have multiple devices, and they install MusicBox across devices 
# --> modify device variable s.t. it captures the case with multiple devices

(980, 3)

In [119]:
play_feature_3 = play[['uid', 'churn']].drop_duplicates()
play_feature_3.shape

(691, 2)

In [134]:
play_uid = play['uid'].drop_duplicates()
play_uid.shape

(691,)

In [140]:
device_num = play[['uid', 'device']].drop_duplicates() \
                                        .groupby('uid')['device'] \
                                        .agg(['max', 'count']) 

device_num.head()

,max,count
uid,,
1668933.0,ar,1
4754114.0,ar,1
7950294.0,ar,1
7950557.0,ar,1
10257342.0,ar,1


In [142]:
device_num.dtypes

max      object
count     int64
dtype: object

In [143]:
def device_adj(row):
    if row['count'] == 1:
        return row['max']
    else:
        return 'both'
    
    
device_num['device'] = device_num.apply(device_adj)
device.head()

KeyError: ('count', 'occurred at index max')

In [145]:
device_num['device'] = np.where(device_num['count'] == 1, device_num['max'], 'both') 

In [156]:
device_num['device'].value_counts()

ar      396
both    289
ip        6
Name: device, dtype: int64

In [157]:
play_feature_4 = play_feature_3.reset_index(drop=True) \
                               .set_index('uid') \
                               .join(device_num['device'])
        
play_feature_4.tail()

,churn,device
uid,,
168810336.0,0,both
158362768.0,1,ar
162111120.0,0,ar
167686400.0,0,ar
167848336.0,0,both


In [158]:
play_feature_all = play_feature_4.join(play_feature, how='left')
play_feature_all.head()

,churn,device,freq_last_30_days_play,mean_last_30_days_play_time,freq_last_14_days_play,mean_last_14_days_play_time,freq_last_7_days_play,mean_last_7_days_play_time,freq_last_3_days_play,mean_last_3_days_play_time,freq_last_1_days_play,mean_last_1_days_play_time
uid,,,,,,,,,,,,
167861376.0,0,both,817,179.506732,263.0,205.908745,18.0,177.444444,NaN,NaN,NaN,NaN
168017424.0,0,both,789,83.002535,306.0,118.669935,302.0,116.804636,232.0,119.133621,175.0,121.828571
168273328.0,0,ar,497,128.913481,261.0,90.444444,261.0,90.444444,261.0,90.444444,182.0,78.274725
168466368.0,1,ar,119,57.848739,49.0,84.530612,NaN,NaN,NaN,NaN,NaN,NaN
168889152.0,0,both,524,168.597328,349.0,177.383954,260.0,186.038462,144.0,201.472222,51.0,247.490196


In [160]:
df_all = play_feature_all.join(search_feature, how='left') \
                         .join(down_feature, how='left') \
                         .fillna(0)

df_all.sample(10)

,churn,device,freq_last_30_days_play,mean_last_30_days_play_time,freq_last_14_days_play,mean_last_14_days_play_time,freq_last_7_days_play,mean_last_7_days_play_time,freq_last_3_days_play,mean_last_3_days_play_time,...,freq_last_30_days_search,freq_last_14_days_search,freq_last_7_days_search,freq_last_3_days_search,freq_last_1_days_search,freq_last_30_days_download,freq_last_14_days_download,freq_last_7_days_download,freq_last_3_days_download,freq_last_1_days_download
uid,,,,,,,,,,,,,,,,,,,,,
168282544.0,0,ar,396,233.247475,174.0,236.178161,173.0,236.364162,26.0,208.192308,...,3.0,2.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
168755024.0,1,both,690,7323.752174,4.0,329.750000,3.0,439.666667,2.0,659.500000,...,9.0,1.0,1.0,1.0,1.0,193.0,0.0,0.0,0.0,0.0
168144880.0,0,ar,428,153.939252,367.0,157.000000,0.0,0.000000,0.0,0.000000,...,37.0,20.0,6.0,0.0,0.0,44.0,32.0,0.0,0.0,0.0
167809936.0,0,both,1112,94.273110,626.0,89.090816,103.0,147.320388,68.0,147.661765,...,93.0,48.0,11.0,6.0,3.0,101.0,16.0,4.0,2.0,1.0
167845360.0,0,ar,2101,185.151832,300.0,140.493333,0.0,0.000000,0.0,0.000000,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168480144.0,0,ar,724,416.255525,410.0,479.673171,75.0,590.106667,15.0,2030.533333,...,104.0,50.0,12.0,0.0,0.0,37.0,9.0,1.0,0.0,0.0
168400928.0,0,both,113,152.035398,13.0,116.846154,12.0,105.666667,3.0,144.666667,...,13.0,8.0,6.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
166936272.0,0,ar,65,18.507692,58.0,12.844828,58.0,12.844828,3.0,115.000000,...,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
167864144.0,0,ar,416,168.189904,114.0,133.982456,53.0,121.320755,10.0,170.000000,...,107.0,51.0,3.0,0.0,0.0,33.0,11.0,3.0,0.0,0.0


# END

In [71]:
df_win = search[(pd.to_datetime('2017-04-29') - search['search_date']).dt.days <= 1]

In [69]:
df_win.tail()

,uid,search_date_str,search_query,search_date
41316,168632256.0,2017-05-12 23:34:36,%e7%9a%87%e5%86%a0,2017-05-12 23:34:36
41317,168632256.0,2017-05-12 23:36:26,%e4%bd%a0%e7%9a%84%e7%b4%a0%e9%a2%9c,2017-05-12 23:36:26
41318,168632256.0,2017-05-12 23:37:08,%e7%9f%ad%e5%81%87%e6%9c%9f,2017-05-12 23:37:08
41319,168632256.0,2017-05-12 23:37:29,%e8%88%8d%e4%b8%8d%e5%be%97%e7%9a%84%e8%83%8c%...,2017-05-12 23:37:29
41320,168632256.0,2017-05-12 23:44:03,%e6%92%95%e5%bf%83%e6%ac%b2,2017-05-12 23:44:03


In [30]:
df_win.groupby('uid')['play_time'].agg(['count', 'sum', 'mean'])

,count,sum,mean
uid,,,
17745172.0,38,4175.0,109.868421
17972352.0,2,367.0,183.500000
19201816.0,32,6256.0,195.500000
47544816.0,5,398.0,79.600000
51908840.0,1,39907.0,39907.000000
96451488.0,58,8963.0,154.534483
132343856.0,101,30093.0,297.950495
148445200.0,37,8821.0,238.405405
150530384.0,25,4201.0,168.040000


In [ ]:
search_1 = compute_freq(search, 'search_date', '2017-04-29', 1)